1. Obtenir la clé API sur Alpha Vantage pour accéder aux données financières.

2. Créer un nouveau répertoire dans le compte GitHub pour héberger le projet Spark-trading.

3. Créer un nouveau notebook dans DataBricks pour exécuter le code Spark-trading.

4. Importer les bibliothèques nécessaires dans ce notebook.

In [0]:
from pyspark.streaming import StreamingContext
from pyspark import SparkContext

import requests

5. Créer une instance du StreamingContext avec la configuration Spark appropriée

In [0]:
# Créer un objet SparkContext
sc = SparkContext(appName="StreamingExample")

# Créer un objet StreamingContext avec un intervalle de traitement spécifié (par exemple, 1 seconde)
#ssc = StreamingContext(sparkContext=sc, batchDuration=1)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-577569076373841>:2
      1 # Créer un objet SparkContext
----> 2 sc = SparkContext(appName="StreamingExample")

File /databricks/spark/python/pyspark/context.py:202, in SparkContext.__init__(self, master, appName, sparkHome, pyFiles, environment, batchSize, serializer, conf, gateway, jsc, profiler_cls, udf_profiler_cls, memory_profiler_cls)
    196 if gateway is not None and gateway.gateway_parameters.auth_token is None:
    197     raise ValueError(
    198         "You are trying to pass an insecure Py4j gateway to Spark. This"
    199         " is not allowed as it is a security risk."
    200     )
--> 202 SparkContext._ensure_initialized(self, gateway=gateway, conf=conf)
    203 try:
    204     self._do_init(
    205         master,
    206         appName,
   (...)
    216         memory_profiler_cls,
    217     )


6. Définir la source de données en utilisant la méthode socketTextStream pour un flux rentrant. Par ex, nous utiliserons la méthode foreachRDD pour télécharger les données d'Alpha Vantage à partir de l'API.

In [0]:
ssc = StreamingContext(sparkContext=sc, batchDuration=1)
# Définir la source de données avec le flux rentrant
lines = ssc.socketTextStream("localhost", 9999)
# Appliquez des transformations sur le flux rentrant
words = lines.flatMap(lambda line: line.split(" "))
wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
# Définir la fonction pour télécharger les données d'Alpha Vantage à partir de l'API

def download_data(rdd):
    # Clé API Alpha Vantage
    api_key = "<JZ3M5S1Y55LBSNSH>"

    # URL de l'API Alpha Vantage pour récupérer les données
    url = 'https://www.alphavantage.co/query"qfunction=ALL_COMMODITIES&interval=monthly&apikey=JZ3M5S1Y55LBSNSH'
    r = requests.get(url)
    data = r.json()

print(data)
# Boucle sur chaque enregistrement du RDD
for record in rdd.collect():
    # Récupération du symbole de la société à partir de l'enregistrement
    symbol = record.strip()
    # Paramètres de la requête
    params = {
        "function": "TIME_SERIES_INTRADAY",
        "symbol": symbol,
        "interval": "1min",
        "apikey": api_key
    }

# Envoi de la requête GET à l'API Alpha Vantage
response = requests.get(url, params=params)

# Votre code pour traiter la réponse et enregistrer les données dans Spark
df = (spark
     .readStream
     .option("maxFilesPerTrigger", 1)
     .format("json")
     .load(events_path)
     )
 
df.isStreaming

# Appliquez la fonction de téléchargement de données à chaque RDD du flux rentrant
#lines.foreachRDD(download_data)
# Appliquez la fonction de téléchargement de données à chaque RDD du flux rentrant
lines.foreachRDD(lambda rdd: rdd.foreach(download_data))


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<command-577569076373844>", line 17, in <module>
    for record in rdd.collect():
NameError: name 'rdd' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 1997, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1112, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1006, in structured_traceback
    return VerboseTB.structured_traceback(
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/ultratb.p

7. Appliquer les opérations de transformation ou d'analyse sur les données téléchargées.

8. Enregistrer les résultats dans des fichiers.

9. Démarrer le traitement du flux en utilisant la méthode start du StreamingContext

In [0]:
ssc.start()

10. Attendez que le traitement soit terminé ou définissez des conditions d'arrêt, puis arrêtez le contexte de streaming :

In [0]:
ssc.awaitTermination()
ssc.stop()

11. Dans GitHub, créer un fichier .txt contenant la clé API d'Alpha Vantage, puis enregistrer-le dans le répertoire approprié.

12. Dans DataBricks, utiliser la méthode read.text() pour lire le fichier contenant la clé API à partir de GitHub.

In [0]:
# Lire le fichier contenant la clé API depuis GitHub
api_key = spark.read.text("https://github.com/orion-alpha-kplr/Spark-trading/api.txt")
